In [ ]:
# general imports
import numpy as np  
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from pymbar import BAR

# Imports from the openff toolkit
from openff.toolkit.topology import Molecule
import torch
from openmm import unit

from neq_functions import neq_from_mm_to_ani, neq_from_ani_to_mm

platform = 'cuda'
num_threads = 2

torch.set_num_threads(num_threads)


In [ ]:
# define simulation constants
time_unit = unit.femtoseconds
energy_unit = unit.kilojoule_per_mole
nr_of_switches=50
n_samples=200
n_steps_per_sample = 2_000
switching_length = 5_001

In [ ]:
# generate a molecule using openff
molecule = Molecule.from_smiles('CC(C(C)=O)C(C)=O', hydrogens_are_explicit=False)
molecule.generate_conformers()

In [ ]:
logavgsumexp_ani_to_mm, stddev_w_ani_to_mm, w_list_ani_to_mm = neq_from_ani_to_mm(molecule, 
                                                                                  n_samples=n_samples, 
                                                                                  n_steps_per_sample=n_steps_per_sample,
                                                                                  nr_of_switches=nr_of_switches,
                                                                                  switching_length=switching_length,
                                                                                  save_samples='data_for_neq/samples_for_ani.pickle')


In [ ]:
logavgsumexp_mm_to_ani, stddev_w_mm_to_ani, w_list_mm_to_ani = neq_from_mm_to_ani(molecule, 
                                                                                  n_samples=n_samples, 
                                                                                  n_steps_per_sample=n_steps_per_sample,
                                                                                  nr_of_switches=nr_of_switches,
                                                                                  switching_length=switching_length)

In [ ]:
plt.hist(w_list_mm_to_ani - np.average(w_list_mm_to_ani))
plt.hist(w_list_ani_to_mm + np.average(w_list_mm_to_ani))
plt.show()


In [ ]:
BAR(np.array(w_list_ani_to_mm), np.array(w_list_mm_to_ani))

In [ ]:
from neq_functions import collect_samples_ani, collect_samples_mm, save_traj, create_mm_sim

ani_samples = collect_samples_ani(molecule, n_samples=100, n_steps_per_sample=100)
mm_samples = collect_samples_mm(None, sim=create_mm_sim(molecule), n_samples=100, n_steps_per_sample=100)

save_traj(ani_samples, molecule, name='ani_samples.dcd')
save_traj(mm_samples, molecule, name='mm_samples.dcd')